In [1]:
%load_ext autoreload
%autoreload 2

import dataclasses

import torch
from torch import nn

In [28]:
from src.network_analysis import count_parameters
from src.networks.skip_nets.dense_skip_net import DenseSkipNet
from src.networks.core.torch_net import TorchNet
from src.networks.core.layer_connections import LayerConnections
from src.networks.skip_nets.additive_skip_net import AdditiveSkipNet

def create_layer(i, last, in_features, out_features):
    l = nn.Linear(in_features, out_features)
    # l.weight.data[:] = 0.1
    # l.bias.data[:] = 0.0
    return nn.Sequential(
        l,
        nn.CELU(),
    )

net = AdditiveSkipNet.from_layer_provider(
    layer_provider=create_layer,
    num_layers=256,
    num_features=512,
    layer_connections='full',
    initial_direct_connection_weight=1.0,
    initial_skip_connection_weight=0.01,
    return_dense=False,
    weights_trainable=False,
)
# net = DenseSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=4,
#     num_features=1,
# )
print(count_parameters(net))
print(net)

net(torch.ones(512) * 0.01)


67239936
AdditiveSkipNet(
  (layers): NetList(
    (0-255): 256 x TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
  )
)


tensor([-2.4459e-02, -1.6072e-02,  9.7477e-04,  4.3461e-02, -4.0533e-03,
         6.5953e-02, -4.7980e-02,  5.0557e-02, -1.9582e-02,  2.7958e-02,
        -1.3671e-02,  1.2019e-02,  1.5130e-02,  1.4208e-02, -3.2068e-03,
        -3.4776e-02, -3.3296e-02,  2.4347e-02,  4.0360e-02,  3.7785e-02,
        -8.7982e-03,  2.6627e-02, -9.5907e-03,  3.2516e-02, -9.1851e-03,
        -5.4405e-02,  4.9836e-03, -6.2159e-02,  2.8282e-03, -3.2337e-02,
         9.6193e-03,  2.6433e-02,  4.6095e-02,  1.1417e-02, -3.7447e-02,
         1.7427e-02, -1.1938e-02,  2.8570e-02,  1.1151e-02, -4.9988e-02,
         4.5225e-03, -8.7485e-04,  3.8626e-04, -2.0135e-02,  2.5093e-02,
        -1.3513e-02,  1.8518e-02,  3.5196e-03,  3.1535e-02,  1.3996e-03,
         1.3780e-02, -2.1670e-02, -5.2225e-03,  2.5584e-02,  4.2089e-03,
         3.8124e-02,  2.6356e-02,  6.5364e-03, -2.6392e-02,  2.5153e-02,
         3.8956e-03, -2.3723e-03,  1.1107e-02, -1.0164e-02, -2.1589e-02,
         4.2959e-04, -5.3201e-03, -7.1018e-03,  3.8

In [32]:
1.61 + 0.1 * (1.3 + 1 + 1 + 1.1)

2.0500000000000003